In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

#import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [4]:
import pandas as pd
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [5]:
train_df = pd.read_csv('../input/dog-breed-identification/labels.csv')

In [6]:
path = "../input/dog-breed-identification/train/"

X = []
for i in range(len(train_df)):
    
    pth = path+train_df.id[i]+".jpg"
    img = np.stack(cv2.resize(cv2.imread(pth), (331, 331))) #reading the image and resizing it.
    X.append(img) 
    

In [7]:
np.shape(X)

In [8]:
#Spliting the data into X an y
X = np.asarray(X)
y = np.asarray(train_df['breed'])

In [9]:
#Onehot encoding the categorical data
y = pd.get_dummies(y)
output_len = len(y.columns)

In [10]:
from tensorflow.keras.layers import Dense, Flatten, Dropout, Conv2D, GlobalAveragePooling2D, BatchNormalization

In [11]:
#calling the pretrained InceptionV3 model from keras

from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input

inputs = tf.keras.Input(shape=(331, 331, 3))
base_model = InceptionV3(weights='imagenet', include_top=False, input_tensor=inputs)
base_model.trainable = False

In [12]:
# passing inputs of one layer to another to create sequence of layers
x = tf.keras.Input(shape=(331,331,3))
x = inputs
 
x = preprocess_input(x)

x = base_model(x, training=False)
x = GlobalAveragePooling2D()(x)
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.2)(x)
x = BatchNormalization()(x)

output = Dense(output_len, activation='softmax')(x)

In [13]:
model = tf.keras.Model(inputs, output)

In [14]:
model.compile(optimizer='Adam', loss="binary_crossentropy", metrics=['Accuracy'])

In [15]:
r = model.fit(X, y, batch_size=32, validation_split=0.2, epochs=15)

In [16]:
train_loss = r.history['loss']
val_loss = r.history['val_loss']

train_acc = r.history['Accuracy']
val_acc = r.history['val_Accuracy']

In [17]:
import matplotlib.pyplot as plt

In [18]:
plt.figure(figsize=(18,6))
plt.plot(train_loss, label='train_los')
plt.plot(val_loss, label='val_loss')
plt.plot(train_acc, label='train_acc')
plt.plot(val_acc, label='val_acc')
plt.xlabel('epochs')
plt.ylabel('value')
plt.legend(loc='upper right')
plt.tight_layout()
plt.show()

#### We can see after 3rd epochs our train accuracy still increasing while val accouracy is almost same. That indicates our model started overfitting.